In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import time

In [4]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
if not os.path.exists(volume_dir):
    os.makedirs(volume_dir)

In [5]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [6]:
scoremaps_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless'

In [7]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']
label_dict = dict([(l,i) for i, l in enumerate(labels)])

# colors = (np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')*255).astype(np.uint8)
# colors[label_dict['BackG']] = 255

In [8]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [4]:
##### reconstruct score volume ######

In [9]:
# stack = 'MD594'
# stack = 'MD593'

for stack in ['MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD585']:

    # build annotation volume
    section_bs_begin, section_bs_end = section_range_lookup[stack]
    print section_bs_begin, section_bs_end

    section_roi_begin, section_roi_end = detect_bbox_range_lookup[stack]
    print section_roi_begin, section_roi_end

    dm = DataManager(stack=stack)

    roi_zmin = int(section_roi_begin*z_xy_ratio_downsampled)
    roi_zmax = int((section_roi_end+1)*z_xy_ratio_downsampled)-1
    print roi_zmin, roi_zmax


    volume_xmin, volume_ymin, volume_zmin = (np.inf, np.inf, np.inf)
    volume_xmax, volume_ymax, volume_zmax = (0,0,0)

    for l in labels:
    # for l in ['VLL']:

        if l == 'BackG': continue

        print l

        volume = np.zeros((dm.image_height/downsample_factor, 
                           dm.image_width/downsample_factor, 
                           int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.float32)
        print volume.shape

        for sec in range(section_roi_begin, section_roi_end+1):

            scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.bp' \
                % {'stack': stack, 'slice': sec, 'label': l}

            if not os.path.exists(scoremap_bp_filepath):
                sys.stderr.write('No scoremap for section %d\n' % sec)
                continue

            print sec

            scoremap_roi = bp.unpack_ndarray_file(scoremap_bp_filepath)

            dataset = stack + '_' + '%04d'%sec + '_roi1'

            interpolation_xmin, interpolation_xmax, \
            interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                                             '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                            {'dataset': dataset, 'label': l})).astype(np.int)

            dense_scoremap_lossless = np.zeros((dm.image_height, dm.image_width), np.float32)

    #         dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
    #                                 interpolation_xmin:interpolation_xmax+1] = scoremap_roi.copy()
    #         scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor].copy()
    #         volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = scoremap[..., np.newaxis].copy()

            dense_scoremap_lossless[interpolation_ymin:interpolation_ymax+1,
                                    interpolation_xmin:interpolation_xmax+1] = scoremap_roi
            scoremap = dense_scoremap_lossless[::downsample_factor, ::downsample_factor]
            volume[:, :, int(sec*z_xy_ratio_downsampled):int((sec+1)*z_xy_ratio_downsampled)] = scoremap[..., np.newaxis].copy()

            del scoremap, scoremap_roi, dense_scoremap_lossless

        ys, xs, zs = np.where(volume > 0)

        xmin = xs.min()
        xmax = xs.max()
        ymin = ys.min()
        ymax = ys.max()
        zmin = zs.min()
        zmax = zs.max()

        print xmin, xmax
        print ymin, ymax
        print zmin, zmax

        if xmin < volume_xmin:
            volume_xmin = xmin
        if ymin < volume_ymin:
            volume_ymin = ymin
        if zmin < volume_zmin:
            volume_zmin = zmin
        if xmax > volume_xmax:
            volume_xmax = xmax
        if ymax > volume_ymax:
            volume_ymax = ymax
        if zmax > volume_zmax:
            volume_zmax = zmax

        bp.pack_ndarray_file(volume[..., roi_zmin:roi_zmax+1],
                     os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % {'stack': stack, 'label': l}))    

        del volume, ys, xs, zs

    print volume_xmin, volume_ymin, volume_zmin
    print volume_xmax, volume_ymax, volume_zmax

    with open(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), 'w') as f:
        f.write((' '.join(['%d']*6)+'\n') % (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax))

91 371
147 319
399 868
5N
(814, 1252, 1233)
147
148
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248

No scoremap for section 149
No scoremap for section 249



250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
88 920
259 731
399 868
7n
(814, 1252, 1233)
147
148
150
151

No scoremap for section 149
No scoremap for section 152



153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196

...

No scoremap for section 251
No scoremap for section 253



254
255
256
257
258
259
260
262

No scoremap for section 261
No scoremap for section 263



264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
286
287
288
289
290
291
292
35 850
291 735
358 795
35 291 358
850 735 795


No scoremap for section 285
